In [1]:
# Generic: A Modular Multi-Pipeline Framework for Probability Fusion Ensembles
# Specific: Cross-Lingual Sentiment Analysis with Probability Fusion Ensembles: A Modular Multi-Pipeline Framework for Low-Resource Languages

import os
import pandas as pd
import pprint

from pathlib import Path
from transformers import pipeline

from src.config import *
from src.metrics import evaluate_pipe
from src import (
    context,
    helper,
    sentiment, 
    utility, 
)

os.environ["TOKENIZERS_PARALLELISM"] = "false"
helper.list_config()

if App.HAS_GPU:
    os.environ["MAMBA_USE_MAMBAPY"] = Mamba.FORCE_CUDA

if App.ACTION == "INFER":
    sample_texts = [
        "Maganda ang serbisyo at mabilis ang delivery!",
        "Sobrang pangit ng karanasan ko.",
        "It was okay, nothing special.",
    ]
    sentiment.infer(sample_texts, Mamba)
    sentiment.infer(sample_texts, MBert)
elif App.ACTION == "ENSEMBLE":
        temps  = [1.1, 0.9]
        weights = [0.4, 0.6]
        ens = sentiment.ensemble([MBert, Mamba], temps, weights)
        print(ens)
elif App.ACTION == "TRAIN":
    mbert_context = context.setup_pipeline(MBert, require_translation = False)
    mbert_trainer = sentiment.train(mbert_context)

    xlmr_context = context.setup_pipeline(Xlmr, require_translation = False)
    xlmr_trainer = sentiment.train(xlmr_context)
    
    mamba_context = context.setup_pipeline(Mamba, require_translation = True)
    mamba_trainer = sentiment.train(mamba_context)
else:
    raise ValueError("Invalid action.")



({'accuracy': 0.3, 'f1_macro': 0.15384615384615385, 'f1_micro': 0.3, 'nll': 1.0986121892929077}, array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0]), array([[0.33333334, 0.33333334, 0.33333334],
       [0.33333334, 0.33333334, 0.33333334],
       [0.33333334, 0.33333334, 0.33333334],
       [0.33333334, 0.33333334, 0.33333334],
       [0.33333334, 0.33333334, 0.33333334],
       [0.33333334, 0.33333334, 0.33333334],
       [0.33333334, 0.33333334, 0.33333334],
       [0.33333334, 0.33333334, 0.33333334],
       [0.33333334, 0.33333334, 0.33333334],
       [0.33333334, 0.33333334, 0.33333334]], dtype=float32))
